In [ ]:
!pip install open3d

In [ ]:
from calibration_functions import pfm_imread, write_pfm
from PIL import Image
from torchvision import transforms

import json
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
with open('../config.json') as f:
    config = json.load(f)
BASE_DIR = config["BASE_DIR"]

# Transformation Functions

In [ ]:
def hflip_image(image):
    return transforms.functional.hflip(image)

def vflip_image(image):
    return transforms.functional.vflip(image)

def rotate_image_180(image):
    return transforms.functional.rotate(image, 180)

def hflip_np(array):
    return np.fliplr(array)

def vflip_np(array):
    return np.flipud(array)

def rotate_np_180(array):
    return np.rot90(array, 2)

def apply_transformation_and_save(files, output_path, image_transform=None, np_transform=None):
    start_index = len(os.listdir(output_path))
    print(f"Starting index for saving files in {output_path}: {start_index + 1}")

    for index, file_path in enumerate(files, start=start_index + 1):
        file_type = 'image' if file_path.endswith('.png') else 'disparity'
        file_name = f'{file_type}_{index:04d}.png' if file_path.endswith('.png') else f'{file_type}_{index:04d}.pfm'
        data, scale = (Image.open(file_path).convert('RGB'), None) if file_path.endswith('.png') else pfm_imread(file_path)

        if file_path.endswith('.png') and image_transform:
            print(f"Applying image transformation to {file_path}")
            data = image_transform(data)
        elif file_path.endswith('.pfm') and np_transform:
            print(f"Applying numpy transformation to {file_path}")
            before_transform = np.array(data, copy=True)
            data = np_transform(data)
            if np.array_equal(data, before_transform):
                print("WARNING: No change in numpy array after transformation.")

        output_file_path = os.path.join(output_path, file_name)
        print(f"Saving {'image' if file_path.endswith('.png') else 'disparity'} as {output_file_path}")

        if file_path.endswith('.png'):
            data.save(output_file_path)
        else:
            write_pfm(output_file_path, data, scale)


# DATASET 1

In [ ]:
left_image_path  = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso/camera1')
right_image_path = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso/camera2')
disparity_path   = os.path.join(BASE_DIR, 'data_1/ground_truth_pfm_rect_cor_disp')
left_images      = sorted([img for img in os.listdir(left_image_path) if img.endswith('.png')])
right_images     = sorted([img for img in os.listdir(right_image_path) if img.endswith('.png')])
disparities   = sorted([img for img in os.listdir(disparity_path) if img.endswith('.pfm')])


In [ ]:
left_output_path      = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso_aug/camera1')
right_output_path     = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso_aug/camera2')
disparity_output_path = os.path.join(BASE_DIR, 'data_1/ground_truth_pfm_rect_cor_disp_aug')
os.makedirs(left_output_path, exist_ok=True)
os.makedirs(right_output_path, exist_ok=True)
os.makedirs(disparity_output_path, exist_ok=True)


In [ ]:
transformations = [
    transforms.Compose([transforms.RandomHorizontalFlip(p=1)]),
    transforms.Compose([transforms.RandomVerticalFlip(p=1)]),
    transforms.Compose([transforms.RandomHorizontalFlip(p=1), transforms.RandomVerticalFlip(p=1)])
]


In [ ]:
left_files = sorted([os.path.join(left_image_path, f) for f in os.listdir(left_image_path) if f.endswith('.png')])[14:]
right_files = sorted([os.path.join(right_image_path, f) for f in os.listdir(right_image_path) if f.endswith('.png')])[14:]
disparity_files = sorted([os.path.join(disparity_path, f) for f in os.listdir(disparity_path) if f.endswith('.pfm')])[14:]

In [ ]:
left_files

In [ ]:
data, _ = pfm_imread(disparity_files[0])
# data = hflip_np(data)
plt.imshow(data)
plt.show()

In [ ]:
apply_transformation_and_save(left_files, left_output_path)
apply_transformation_and_save(right_files, right_output_path)
apply_transformation_and_save(disparity_files, disparity_output_path)


In [ ]:
transformations = [
    (hflip_image, hflip_np),
    (vflip_image, vflip_np),
    (rotate_image_180, rotate_np_180)
]

# apply_transformation_and_save(left_files, left_output_path, hflip_image, hflip_np)
# apply_transformation_and_save(right_files, right_output_path, hflip_image, hflip_np)
# apply_transformation_and_save(disparity_files, disparity_output_path, hflip_image, hflip_np)

apply_transformation_and_save(left_files, left_output_path, vflip_image, vflip_np)
apply_transformation_and_save(right_files, right_output_path, vflip_image, vflip_np)
apply_transformation_and_save(disparity_files, disparity_output_path, vflip_image, vflip_np)

# apply_transformation_and_save(left_files, left_output_path, rotate_image_180, rotate_np_180)
# apply_transformation_and_save(right_files, right_output_path, rotate_image_180, rotate_np_180)
# apply_transformation_and_save(disparity_files, disparity_output_path, rotate_image_180, rotate_np_180)

# DATASET 2

In [ ]:
left_image_path  = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso/camera1')
right_image_path = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso/camera2')
disparity_path   = os.path.join(BASE_DIR, 'data_2/ground_truth_pfm_rect_cor_disp')
left_images      = sorted([img for img in os.listdir(left_image_path) if img.endswith('.png')])
right_images     = sorted([img for img in os.listdir(right_image_path) if img.endswith('.png')])
disparities   = sorted([img for img in os.listdir(disparity_path) if img.endswith('.pfm')])

In [ ]:
disparity_path

In [ ]:
left_output_path      = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso_aug/camera1')
right_output_path     = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso_aug/camera2')
disparity_output_path = os.path.join(BASE_DIR, 'data_2/ground_truth_pfm_rect_cor_disp_aug')
os.makedirs(left_output_path, exist_ok=True)
os.makedirs(right_output_path, exist_ok=True)
os.makedirs(disparity_output_path, exist_ok=True)


In [ ]:
transformations = [
    transforms.Compose([transforms.RandomHorizontalFlip(p=1)]),
    transforms.Compose([transforms.RandomVerticalFlip(p=1)]),
    transforms.Compose([transforms.RandomHorizontalFlip(p=1), transforms.RandomVerticalFlip(p=1)])
]

In [ ]:
left_files = sorted([os.path.join(left_image_path, f) for f in os.listdir(left_image_path) if f.endswith('.png')])
right_files = sorted([os.path.join(right_image_path, f) for f in os.listdir(right_image_path) if f.endswith('.png')])
disparity_files = sorted([os.path.join(disparity_path, f) for f in os.listdir(disparity_path) if f.endswith('.pfm')])

In [ ]:
apply_transformation_and_save(left_files, left_output_path)
apply_transformation_and_save(right_files, right_output_path)
apply_transformation_and_save(disparity_files, disparity_output_path)


In [ ]:
transformations = [
    (hflip_image, hflip_np),
    (vflip_image, vflip_np),
    (rotate_image_180, rotate_np_180)
]

# apply_transformation_and_save(left_files, left_output_path, hflip_image, hflip_np)
# apply_transformation_and_save(right_files, right_output_path, hflip_image, hflip_np)
# apply_transformation_and_save(disparity_files, disparity_output_path, hflip_image, hflip_np)

apply_transformation_and_save(left_files, left_output_path, vflip_image, vflip_np)
apply_transformation_and_save(right_files, right_output_path, vflip_image, vflip_np)
apply_transformation_and_save(disparity_files, disparity_output_path, vflip_image, vflip_np)

# apply_transformation_and_save(left_files, left_output_path, rotate_image_180, rotate_np_180)
# apply_transformation_and_save(right_files, right_output_path, rotate_image_180, rotate_np_180)
# apply_transformation_and_save(disparity_files, disparity_output_path, rotate_image_180, rotate_np_180)

# DATASET 1 HALF

In [ ]:
left_image_path  = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso_half/camera1')
right_image_path = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso_half/camera2')
disparity_path   = os.path.join(BASE_DIR, 'data_1/ground_truth_pfm_rect_cor_disp_half')
left_images      = sorted([img for img in os.listdir(left_image_path) if img.endswith('.png')])
right_images     = sorted([img for img in os.listdir(right_image_path) if img.endswith('.png')])
disparities   = sorted([img for img in os.listdir(disparity_path) if img.endswith('.pfm')])

In [ ]:
disparity_path

In [ ]:
left_output_path      = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso_half_aug/camera1')
right_output_path     = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso_half_aug/camera2')
disparity_output_path = os.path.join(BASE_DIR, 'data_1/ground_truth_pfm_rect_cor_disp_half_aug')
os.makedirs(left_output_path, exist_ok=True)
os.makedirs(right_output_path, exist_ok=True)
os.makedirs(disparity_output_path, exist_ok=True)

In [ ]:
transformations = [
    transforms.Compose([transforms.RandomHorizontalFlip(p=1)]),
    transforms.Compose([transforms.RandomVerticalFlip(p=1)]),
    transforms.Compose([transforms.RandomHorizontalFlip(p=1), transforms.RandomVerticalFlip(p=1)])
]

In [ ]:
left_files = sorted([os.path.join(left_image_path, f) for f in os.listdir(left_image_path) if f.endswith('.png')])[14:]
right_files = sorted([os.path.join(right_image_path, f) for f in os.listdir(right_image_path) if f.endswith('.png')])[14:]
disparity_files = sorted([os.path.join(disparity_path, f) for f in os.listdir(disparity_path) if f.endswith('.pfm')])[14:]

In [ ]:
apply_transformation_and_save(left_files, left_output_path)
apply_transformation_and_save(right_files, right_output_path)
apply_transformation_and_save(disparity_files, disparity_output_path)

In [ ]:
# Apply transformations
transformations = [
    (hflip_image, hflip_np),
    (vflip_image, vflip_np),
    (rotate_image_180, rotate_np_180)
]

# apply_transformation_and_save(left_files, left_output_path, hflip_image, hflip_np)
# apply_transformation_and_save(right_files, right_output_path, hflip_image, hflip_np)
# apply_transformation_and_save(disparity_files, disparity_output_path, hflip_image, hflip_np)

apply_transformation_and_save(left_files, left_output_path, vflip_image, vflip_np)
apply_transformation_and_save(right_files, right_output_path, vflip_image, vflip_np)
apply_transformation_and_save(disparity_files, disparity_output_path, vflip_image, vflip_np)

# apply_transformation_and_save(left_files, left_output_path, rotate_image_180, rotate_np_180)
# apply_transformation_and_save(right_files, right_output_path, rotate_image_180, rotate_np_180)
# apply_transformation_and_save(disparity_files, disparity_output_path, rotate_image_180, rotate_np_180)

# DATASET 2 HALF

In [ ]:
left_image_path  = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso_half/camera1')
right_image_path = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso_half/camera2')
disparity_path   = os.path.join(BASE_DIR, 'data_2/ground_truth_pfm_rect_cor_disp_half')
left_images      = sorted([img for img in os.listdir(left_image_path) if img.endswith('.png')])
right_images     = sorted([img for img in os.listdir(right_image_path) if img.endswith('.png')])
disparities   = sorted([img for img in os.listdir(disparity_path) if img.endswith('.pfm')])

In [ ]:
disparity_path

In [ ]:
left_output_path      = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso_half_aug/camera1')
right_output_path     = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso_half_aug/camera2')
disparity_output_path = os.path.join(BASE_DIR, 'data_2/ground_truth_pfm_rect_cor_disp_half_aug')
os.makedirs(left_output_path, exist_ok=True)
os.makedirs(right_output_path, exist_ok=True)
os.makedirs(disparity_output_path, exist_ok=True)

In [ ]:
transformations = [
    transforms.Compose([transforms.RandomHorizontalFlip(p=1)]),
    transforms.Compose([transforms.RandomVerticalFlip(p=1)]),
    transforms.Compose([transforms.RandomHorizontalFlip(p=1), transforms.RandomVerticalFlip(p=1)])
]

In [ ]:
left_files = sorted([os.path.join(left_image_path, f) for f in os.listdir(left_image_path) if f.endswith('.png')])
right_files = sorted([os.path.join(right_image_path, f) for f in os.listdir(right_image_path) if f.endswith('.png')])
disparity_files = sorted([os.path.join(disparity_path, f) for f in os.listdir(disparity_path) if f.endswith('.pfm')])

In [ ]:
apply_transformation_and_save(left_files, left_output_path)
apply_transformation_and_save(right_files, right_output_path)
apply_transformation_and_save(disparity_files, disparity_output_path)


In [ ]:
# Apply transformations
transformations = [
    (hflip_image, hflip_np),
    (vflip_image, vflip_np)
    (rotate_image_180, rotate_np_180)
]

# apply_transformation_and_save(left_files, left_output_path, hflip_image, hflip_np)
# apply_transformation_and_save(right_files, right_output_path, hflip_image, hflip_np)
# apply_transformation_and_save(disparity_files, disparity_output_path, hflip_image, hflip_np)

apply_transformation_and_save(left_files, left_output_path, vflip_image, vflip_np)
apply_transformation_and_save(right_files, right_output_path, vflip_image, vflip_np)
apply_transformation_and_save(disparity_files, disparity_output_path, vflip_image, vflip_np)

# apply_transformation_and_save(left_files, left_output_path, rotate_image_180, rotate_np_180)
# apply_transformation_and_save(right_files, right_output_path, rotate_image_180, rotate_np_180)
# apply_transformation_and_save(disparity_files, disparity_output_path, rotate_image_180, rotate_np_180)

# VISUALIZE THE AUGMENTED IMAGES

In [ ]:
left_check_path      = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso_aug/camera1')
right_check_path     = os.path.join(BASE_DIR, 'data_1/multicast_rect_reso_aug/camera2')
disparity_check_path = os.path.join(BASE_DIR, 'data_1/ground_truth_pfm_rect_cor_disp_aug')

left_files = sorted([os.path.join(left_check_path, f) for f in os.listdir(left_check_path) if f.endswith('.png')])
right_files = sorted([os.path.join(right_check_path, f) for f in os.listdir(right_check_path) if f.endswith('.png')])
disparity_files = sorted([os.path.join(disparity_check_path, f) for f in os.listdir(disparity_check_path) if f.endswith('.pfm')])

for count, (left_file, right_file, disparity_file) in enumerate(zip(left_files, right_files, disparity_files)):
    left_image = Image.open(left_file)
    right_image = Image.open(right_file)
    disparity, _ = pfm_imread(disparity_file)

    plt.figure(figsize=(15, 5), dpi=100)
    plt.subplot(131)
    plt.imshow(disparity, cmap='viridis')

    plt.subplot(132)
    plt.imshow(left_image)

    plt.subplot(133)
    plt.imshow(right_image)

    plt.savefig(f'check_augmentation/plot_{count:03d}.png')
